# Diffusion in 1D

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Optimal interpolation



In [2]:
@interact
def OI_Kalman(h_function=['exponential','gaussian'],h_scale=(1.0,5.0,1.0), obs_std_1=(0.1,0.2,0.005),obs_std_2=(0.1,0.2,0.005), model_std=(0.1,0.2,0.005)):
    n = 50
    x = np.linspace(0, 50, n)
    nobs = 2
    H = np.zeros(shape=(nobs,n))
    H[0,20] = 1
    H[1,40] = 1
    
    C = np.zeros(shape=(n,n))

    for i in range(n):
        for j in range(n):
            dist = np.abs(x[i] - x[j])
            if h_function=='exponential':
                w = np.exp(-dist/h_scale)
            else:
                w = np.exp(-1/2*(dist**2/h_scale**2))
            C[i,j] = w

    P = model_std**2*C

    R  = np.diag([obs_std_1**2,obs_std_2**2])
    
    PR = (H@P@H.T + R)
    
    # diagonal R
    PRinv = PR.copy()
    np.fill_diagonal(PRinv, 1/np.diagonal(PR)) 

    K = P@H.T @ PRinv
    

    fig, (ax1, ax2) = plt.subplots(ncols=2,figsize=(12,4))
    ax1.plot(x,K)
    ax1.set_ylim((0,1))
    ax1.set_title("K: kalman gain")
    ax1.set_xlabel("x")
    
    ax2.imshow(P,vmin=0,vmax=0.05)
    ax2.set_title("P: model covariance")

interactive(children=(Dropdown(description='h_function', options=('exponential', 'gaussian'), value='exponenti…

## Data assimilation

In [3]:
obs = np.array([1.        , 0.39430239, 0.28454729, 0.23153953, 0.19628773,
       0.16936845, 0.1474706 , 0.12907749, 0.11334738, 0.09974623,
       0.08790242])

@interact
def diffuse(scheme=['No DA', 'Direct insertion', 'OI'],
            h_function=['exponential','gaussian'],
            h_scale=(1.0,5.0,1.0),
            diffusivity=(0.01, 0.1, 0.005),
            model_std=(0.01,0.05,0.01),
            obs_std=(0.01,0.05,0.01),
            left_no_flux_boundary=False, right_no_flux_boundary=False,show_obs=False):

    a = diffusivity
    Nx = 50
    Nt = 10001
    t = np.linspace(0, 1, Nt)
    x = np.linspace(0, 50, Nx)
    Nout = (Nt // 1000) + 2
    tout = np.linspace(0, 1, Nout-1)

    T0 = np.zeros(Nx, dtype=float)
    
    T0[17:25] = 1.
    T2 = T0.copy()

    do_me          = np.ones_like(T0, dtype=bool)
    
    #if not no_flux_boundary:
    do_me[[0, -1]] = False    #  keep the boundaries of your bounding box fixed

    res = np.empty(shape=(Nout,Nx))
    ts = np.empty(shape=(Nt,Nx))
    res[0] = T0.copy()
    kernel = np.array([a, (1 - 2.*a), a]) # diffusion kernel

    iout = 1

    n = 50
    nobs = 1
    H = np.zeros(shape=(nobs,n))
    H[0,20] = 1
    
    P = np.zeros(shape=(n,n))

    for i in range(n):
        for j in range(n):
            dist = np.abs(x[i] - x[j])
            if h_function=='exponential':
                w = np.exp(-dist/h_scale)
            else:
                w = np.exp(-1/2*(dist**2/h_scale**2))
            P[i,j] = model_std**2*w

    R  = np.diag(np.repeat(obs_std**2,nobs))
    
    PR = (H@P@H.T + R)
    
    # diagonal R
    PRinv = PR.copy()
    np.fill_diagonal(PRinv, 1/np.diagonal(PR)) 

    K = P@H.T @ PRinv

    for i in range(Nt):
        Ttemp      = convolve(T2, kernel)
        T2[do_me]  = Ttemp[do_me]
        
        if left_no_flux_boundary:
            T2[0] = T2[1]

        if right_no_flux_boundary:
            T2[-1] = T2[-2]

        ts[i] = T2.copy()
        if not i%1000:
            if scheme == "Direct insertion":
                T2[20] = obs[iout-1]
            elif scheme == "OI":
                x_f = T2
                y = obs[iout-1]
                
                # Update equation
                x_a =  x_f + K@(y - H@x_f)
                
                T2 = x_a

            res[iout]= T2.copy()
            iout+=1

        
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(16,6))
    #for i in range(Nout):
    #    thing = res[i,:]
    ax1.plot(res[0],label="Initial")
    ax1.plot(res[2],label="tout=2")
    ax1.plot(res[5],label="tout=5")
    ax1.plot(res[-1],label="End")
    ax1.set_xlabel("Space")
    ax1.legend()

    ax2.plot(t,ts[:,10],label="x=10")
    ax2.plot(t,ts[:,20],label="x=20")
    ax2.plot(t,ts[:,40],label="x=40")
    if show_obs:
        ax2.scatter(tout,obs, label="Observation (x=20)")
    ax2.set_xlabel("Time")
    ax2.legend()

interactive(children=(Dropdown(description='scheme', options=('No DA', 'Direct insertion', 'OI'), value='No DA…